In [ ]:
import os
import sys
import math
import cv2
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from collections import Counter
import imutils
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import model_from_json
from tensorflow.keras.backend import resize_images

In [ ]:
count = 0
X_b_canny=[]
X_img_hist=[]
X_denoised=[]
X_normal=[]
label = []
# alph = [10,11,23,24]
for i in range(24):
    if i==9:
        continue
    s = 'PATH TO DATA' + chr(ord('A') + i)
    for filename in os.listdir(s.strip()):
        if filename.endswith(".jpg"):
            img = cv2.imread(s + "/" + filename)
#             img = img[...,::-1] # Added
            normal = img[...,::-1]
            img = cv2.cvtColor(img ,cv2.COLOR_BGR2YUV)
            img[:,:,0] = cv2.equalizeHist(img[:,:,0])
            img_hist = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
            denoised = cv2.bilateralFilter(img_hist, 7, 100, 100) 
            
            v = np.median(denoised)
            lower = int(max(0, (1.0 - 0.33) * v))
            upper = int(min(255, (1.0 + 0.33) * v))
            img_blur_canny = cv2.Canny(denoised,lower,upper)
            
            X_normal.append(normal)
            X_b_canny.append(img_blur_canny)
            X_img_hist.append(img_hist)
            X_denoised.append(denoised)
            label.append(i)
            count +=1
    print(s,count)   
print(count)
X_normal = np.asarray(X_normal)
X_b_canny = np.asarray(X_b_canny)
X_denoised = np.asarray(X_denoised)
X_img_hist = np.asarray(X_img_hist)
Y = np.asarray(label)
print(Y.shape)
print(X_normal.shape)
print(X_b_canny.shape)
print(X_denoised.shape)
print(X_img_hist.shape)
np.savez_compressed("data_all.npz", X_normal, X_b_canny,X_denoised,X_img_hist,Y)